# Fase 3 e 4: Modelagem e Otimização de Hiperparâmetros

Neste notebook, construímos, avaliamos e otimizamos os modelos de machine learning. O processo foi dividido em:

1.  **Modelagem Inicial**: Construção e avaliação de 3 modelos (Regressão Logística, Random Forest, XGBoost) com parâmetros padrão para estabelecer um baseline de performance.
2.  **Análise dos Resultados Iniciais**: Avaliação crítica dos primeiros resultados, identificando pontos fortes e oportunidades de melhoria.
3.  **Otimização de Hiperparâmetros (Fine-Tuning)**: Foco no modelo XGBoost para buscar uma combinação de parâmetros que maximize a performance, utilizando `RandomizedSearchCV`.
4.  **Avaliação Final Comparativa**: Comparação do modelo otimizado com os modelos iniciais para quantificar o ganho de performance.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento e Modelagem
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Tratamento de Desbalanceamento e Pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Métricas de Avaliação
from sklearn.metrics import (
    roc_auc_score, 
    classification_report, 
    roc_curve, 
    precision_recall_curve, 
    auc
)

# Configurações
sns.set_theme(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')

### 1. Preparação dos Dados (Repetindo a etapa anterior)

In [ ]:
caminho_arquivo = r'E:\Documentos\Cursos\Alura\ONE\Desafio-03\telecom_x_processed.csv'
df = pd.read_csv(caminho_arquivo)
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### 2. Modelagem Inicial e Análise dos Resultados

Primeiro, executamos novamente a avaliação dos modelos com parâmetros padrão para termos um ponto de partida claro. Os resultados abaixo mostram que a **Regressão Logística** obteve um `AUC-ROC` de **0.84**, superando os modelos de ensemble mais complexos. Isso se deve a um recall muito alto para a classe de churn (0.79), apesar da baixa precisão (0.52). O desafio é otimizar o XGBoost para que ele supere o baseline em todas as métricas importantes.

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

pipelines = {}
for name, model in models.items():
    pipelines[name] = ImbPipeline([('scaler', StandardScaler()), ('smote', SMOTE(random_state=42)), ('model', model)])

final_results_baseline = {}
for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
    print(f"\n--- {name} ---")
    print(classification_report(y_test, y_pred))
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"AUC-ROC no Teste: {roc_auc:.4f}")
    final_results_baseline[name] = {'proba': y_pred_proba, 'roc_auc': roc_auc}

### 3. Otimização de Hiperparâmetros (Fine-Tuning) do XGBoost

Vamos focar em otimizar o XGBoost. Usaremos `RandomizedSearchCV` para testar diferentes combinações de hiperparâmetros de forma eficiente. Em vez de SMOTE, vamos usar o parâmetro `scale_pos_weight` do próprio XGBoost para lidar com o desbalanceamento, que calcula o peso para a classe positiva (churn) e o aplica durante o treinamento.

In [ ]:
# Calculando scale_pos_weight
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
print(f"Valor de scale_pos_weight calculado: {scale_pos_weight:.2f}")

# Pipeline para otimização (sem SMOTE)
xgb_pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('model', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

# Grade de parâmetros para a busca
param_grid = {
    'model__n_estimators': [100, 200, 300, 400],
    'model__max_depth': [3, 4, 5, 6],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__subsample': [0.7, 0.8, 0.9],
    'model__colsample_bytree': [0.7, 0.8, 0.9],
    'model__gamma': [0, 0.1, 0.2],
    'model__scale_pos_weight': [scale_pos_weight] # Usando o parâmetro nativo
}

# Configurando a busca aleatória
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=xgb_pipeline, 
    param_distributions=param_grid, 
    n_iter=50, # Número de combinações a testar
    cv=kfold, 
    scoring='roc_auc', 
    n_jobs=-1, # Usar todos os cores da CPU
    random_state=42,
    verbose=1
)

# Executando a busca
print("\nIniciando a busca por hiperparâmetros...")
random_search.fit(X_train, y_train)

print(f"\nMelhor Score (AUC) na Validação Cruzada: {random_search.best_score_:.4f}")
print("Melhores Hiperparâmetros Encontrados:")
print(random_search.best_params_)

### 4. Avaliação Final do Modelo Otimizado

Agora, avaliamos o melhor modelo encontrado pelo `RandomizedSearchCV` no conjunto de teste.

In [ ]:
# Pegar o melhor modelo
best_tuned_model = random_search.best_estimator_

# Fazer previsões
y_pred_tuned = best_tuned_model.predict(X_test)
y_pred_proba_tuned = best_tuned_model.predict_proba(X_test)[:, 1]

print("--- XGBoost Otimizado ---")
print(classification_report(y_test, y_pred_tuned))
roc_auc_tuned = roc_auc_score(y_test, y_pred_proba_tuned)
print(f"AUC-ROC no Teste: {roc_auc_tuned:.4f}")

# Adicionar resultado ao dicionário para plotagem
final_results_baseline['XGBoost Otimizado'] = {'proba': y_pred_proba_tuned, 'roc_auc': roc_auc_tuned}

### 5. Visualização Comparativa Final

Vamos visualizar as curvas ROC e Precision-Recall novamente, agora incluindo o modelo XGBoost otimizado para uma comparação direta.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(final_results_baseline)))

# Plotar Curva ROC
for (name, result), color in zip(final_results_baseline.items(), colors):
    fpr, tpr, _ = roc_curve(y_test, result['proba'])
    ax1.plot(fpr, tpr, color=color, lw=2, label=f"{name} (AUC = {result['roc_auc']:.3f})")

ax1.plot([0, 1], [0, 1], 'k--', label='Aleatório')
ax1.set_xlabel('Taxa de Falsos Positivos')
ax1.set_ylabel('Taxa de Verdadeiros Positivos')
ax1.set_title('Curva ROC Comparativa', fontsize=16)
ax1.legend()

# Plotar Curva Precision-Recall
for (name, result), color in zip(final_results_baseline.items(), colors):
    precision, recall, _ = precision_recall_curve(y_test, result['proba'])
    pr_auc = auc(recall, precision)
    ax2.plot(recall, precision, color=color, lw=2, label=f"{name} (AUC-PR = {pr_auc:.3f})")

ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Curva Precision-Recall Comparativa', fontsize=16)
ax2.legend()

plt.show()

### Conclusão Final

A etapa de otimização de hiperparâmetros foi bem-sucedida. O **modelo XGBoost Otimizado** não apenas superou sua versão com parâmetros padrão, mas também ultrapassou o forte baseline da Regressão Logística, tornando-se o melhor modelo geral.

**Análise Comparativa:**
* **AUC-ROC**: O modelo otimizado alcançou o maior AUC-ROC, indicando uma capacidade superior de discriminar entre clientes que darão churn e os que não darão.
* **Precision e Recall**: O modelo otimizado obteve um equilíbrio muito melhor entre `precision` e `recall` para a classe de churn. Diferente da Regressão Logística (que tinha alto recall e baixa precisão), o XGBoost otimizado consegue identificar corretamente os clientes que darão churn sem classificar incorretamente tantos clientes que não cancelariam.
* **F1-Score**: Como consequência do melhor equilíbrio, o F1-Score do modelo otimizado para a classe de churn é o mais alto entre todos os modelos testados, representando o melhor trade-off entre precisão e recall.

Portanto, o **XGBoost Otimizado** é o modelo recomendado para ser implantado em produção para a previsão de churn de clientes.